In [2]:
pip install requests schedule matplotlib


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import sqlite3
import time
import schedule
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Replace with your OpenWeatherMap API key
API_KEY = '4d7dd4609890555469e8aecd0c818893'
cities = ["Delhi", "Mumbai", "Chennai", "Bangalore", "Kolkata", "Hyderabad"]
base_url = "https://api.openweathermap.org/data/2.5/weather?q="

# Database setup
conn = sqlite3.connect('weather_data.db')
c = conn.cursor()

# Create a table to store weather data if it doesn't exist
c.execute('''CREATE TABLE IF NOT EXISTS weather (
    city TEXT, 
    timestamp TEXT, 
    temp REAL, 
    feels_like REAL, 
    weather_main TEXT, 
    temp_min REAL, 
    temp_max REAL)''')

# Commit and close DB connection
conn.commit()

# Function to convert Kelvin to Celsius
def kelvin_to_celsius(kelvin):
    return kelvin - 273.15

# Function to fetch weather data from the API for a given city
def fetch_weather(city):
    params = {
        'q': city,
        'appid': API_KEY
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return {
            'city': city,
            'timestamp': datetime.utcfromtimestamp(data['dt']).strftime('%Y-%m-%d %H:%M:%S'),
            'temp': kelvin_to_celsius(data['main']['temp']),
            'feels_like': kelvin_to_celsius(data['main']['feels_like']),
            'weather_main': data['weather'][0]['main'],
            'temp_min': kelvin_to_celsius(data['main']['temp_min']),
            'temp_max': kelvin_to_celsius(data['main']['temp_max'])
        }
    else:
        print(f"Failed to fetch data for {city}. Status code: {response.status_code}")
        return None

# Function to store weather data in the database
def store_weather_data(weather_data):
    conn = sqlite3.connect('weather_data.db')
    c = conn.cursor()
    c.execute('''INSERT INTO weather (city, timestamp, temp, feels_like, weather_main, temp_min, temp_max) 
                 VALUES (?, ?, ?, ?, ?, ?, ?)''', 
              (weather_data['city'], weather_data['timestamp'], weather_data['temp'], weather_data['feels_like'],
               weather_data['weather_main'], weather_data['temp_min'], weather_data['temp_max']))
    conn.commit()
    conn.close()

# Function to fetch and store weather data for all cities
def update_weather():
    for city in cities:
        weather_data = fetch_weather(city)
        if weather_data:
            store_weather_data(weather_data)
            print(f"Stored weather data for {city}")

# Function to compute daily summary (average, min, max temperatures and dominant weather)
def daily_summary():
    conn = sqlite3.connect('weather_data.db')
    c = conn.cursor()

    today = datetime.now().strftime('%Y-%m-%d')
    c.execute(f"SELECT city, AVG(temp), MAX(temp), MIN(temp), weather_main FROM weather WHERE timestamp LIKE '{today}%' GROUP BY city")
    rows = c.fetchall()

    print(f"\nDaily Summary for {today}:\n")
    for row in rows:
        city, avg_temp, max_temp, min_temp, weather_main = row
        print(f"City: {city} | Avg Temp: {avg_temp:.2f}°C | Max Temp: {max_temp:.2f}°C | Min Temp: {min_temp:.2f}°C | Dominant Weather: {weather_main}")

    conn.close()

# Function to check for threshold alerts (e.g., temp > 35°C for 2 consecutive updates)
def check_alerts():
    threshold_temp = 35.0  # You can change this value
    conn = sqlite3.connect('weather_data.db')
    c = conn.cursor()

    for city in cities:
        c.execute(f"SELECT temp FROM weather WHERE city = ? ORDER BY timestamp DESC LIMIT 2", (city,))
        temps = c.fetchall()
        if len(temps) == 2 and all(temp[0] > threshold_temp for temp in temps):
            print(f"Alert: Temperature in {city} exceeded {threshold_temp}°C for two consecutive updates!")

    conn.close()

# Function to schedule daily tasks
def run_scheduler():
    # Update weather every 5 minutes
    schedule.every(5).minutes.do(update_weather)
    # Check alerts every 5 minutes
    schedule.every(5).minutes.do(check_alerts)
    # Daily summary at 11:59 PM
    schedule.every().day.at("23:59").do(daily_summary)

    while True:
        schedule.run_pending()
        time.sleep(1)

# Plot the temperature trends for the day
def plot_temp_trends(city):
    conn = sqlite3.connect('weather_data.db')
    c = conn.cursor()

    today = datetime.now().strftime('%Y-%m-%d')
    c.execute(f"SELECT timestamp, temp FROM weather WHERE city = ? AND timestamp LIKE '{today}%' ORDER BY timestamp", (city,))
    rows = c.fetchall()

    if rows:
        timestamps = [datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S') for row in rows]
        temps = [row[1] for row in rows]

        plt.plot(timestamps, temps, marker='o')
        plt.title(f'Temperature Trend for {city} on {today}')
        plt.xlabel('Time')
        plt.ylabel('Temperature (°C)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print(f"No data available for {city} today.")

    conn.close()

# Entry point
if __name__ == "__main__":
    run_scheduler()
